In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from timm import create_model
from tqdm import tqdm
from PIL import Image
import os
from pathlib import Path
import urllib.request
import zipfile
import shutil
import random
import numpy as np

In [ ]:
# 랜덤 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Random seed set to {seed}")

set_seed(42)

Random seed set to 42


In [ ]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# PACS 데이터셋 자동 다운로드 (Kaggle Hub 사용)
def download_pacs():
    """Kaggle Hub를 이용한 PACS 데이터셋 다운로드"""
    try:
        import kagglehub
        print("Downloading PACS dataset from Kaggle...")
        path = kagglehub.dataset_download("nickfratto/pacs-dataset")
        print(f"Path to dataset files: {path}")
        return Path(path)
    except ImportError:
        print("kagglehub not installed. Installing...")
        os.system("pip install kagglehub")
        import kagglehub
        path = kagglehub.dataset_download("nickfratto/pacs-dataset")
        print(f"Path to dataset files: {path}")
        return Path(path)
    except Exception as e:
        print(f"Error downloading PACS: {e}")
        print("\nPlease install kagglehub and try again:")
        print("pip install kagglehub")
        raise

In [ ]:
# PACS 데이터셋 클래스
class PACSDataset(Dataset):
    def __init__(self, root_dir, domains, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []
        self.classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        # 데이터 로드
        for domain in domains:
            # 여러 가능한 경로 패턴 시도
            possible_paths = [
                self.root_dir / domain,
                self.root_dir / 'PACS' / domain,
                self.root_dir / 'kfold' / domain,
                self.root_dir / 'pacs' / domain,
                # Kaggle 데이터셋의 실제 구조
                self.root_dir / 'dct2_images' / 'dct2_images' / domain,
                # train/val 구조가 있을 수 있음
                self.root_dir / 'train' / domain,
                self.root_dir / 'val' / domain,
            ]

            domain_path = None
            for path in possible_paths:
                if path.exists():
                    domain_path = path
                    print(f"Found domain '{domain}' at: {path}")
                    break

            if domain_path is None:
                print(f"Warning: Domain {domain} not found in any expected location")
                continue

            for class_name in self.classes:
                class_path = domain_path / class_name
                if class_path.exists():
                    # jpg, png, jpeg 등 다양한 이미지 형식 지원
                    for ext in ['*.jpg', '*.png', '*.jpeg', '*.JPEG', '*.JPG']:
                        for img_file in class_path.glob(ext):
                            self.samples.append((str(img_file), self.class_to_idx[class_name], domain))

        if len(self.samples) == 0:
            print(f"Warning: No samples found for domains: {domains}")
        else:
            print(f"Loaded {len(self.samples)} samples from domains: {domains}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label, domain = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# 하이퍼파라미터
batch_size = 32
num_epochs = 30
learning_rate = 0.0001
num_classes = 7
weight_decay = 0.0005

In [ ]:
# PACS 도메인
all_domains = ['photo', 'art_painting', 'cartoon', 'sketch']

# 데이터 전처리
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# 학습 함수
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({'loss': f'{running_loss/(pbar.n+1):.4f}',
                         'acc': f'{100.*correct/total:.2f}%'})

    return running_loss/len(loader), 100.*correct/total

In [ ]:
# 평가 함수
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc='Evaluating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss/len(loader), 100.*correct/total

In [ ]:
# Domain Generalization 실험
def run_domain_generalization(data_root, test_domain):
    """
    Leave-one-domain-out 방식으로 실험 수행
    """
    print(f"\n{'='*80}")
    print(f"Testing on domain: {test_domain}")
    print(f"Training on domains: {[d for d in all_domains if d != test_domain]}")
    print(f"{'='*80}\n")

    # 학습/테스트 도메인 분리
    train_domains = [d for d in all_domains if d != test_domain]

    # 데이터셋 생성
    train_dataset = PACSDataset(data_root, train_domains, transform=transform_train)
    test_dataset = PACSDataset(data_root, [test_domain], transform=transform_test)

    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print(f"Error: No samples found. Please check your data path.")
        return None, None
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True, num_workers=4, pin_memory=True,
                            drop_last=True,  # 마지막 불완전한 배치 제거
                            worker_init_fn=lambda worker_id: np.random.seed(42 + worker_id))
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                           shuffle=False, num_workers=4, pin_memory=True)

    # 모델 생성
    model = create_model('mobilenetv4_conv_small.e2400_r224_in1k',
                        pretrained=True, num_classes=num_classes)
    model = model.to(device)

    # 손실 함수와 옵티마이저
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    # 학습
    best_acc = 0.0
    results = []

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 60)

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        scheduler.step()

        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

        results.append({
            'epoch': epoch+1,
            'train_acc': train_acc,
            'test_acc': test_acc
        })

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f'best_model_{test_domain}.pth')
            print(f'✓ Best model saved! Accuracy: {best_acc:.2f}%')

    return best_acc, results

In [ ]:
# 메인 실험 실행
if __name__ == "__main__":
    # PACS 데이터셋 Kaggle에서 다운로드
    try:
        data_root = download_pacs()
        print(f"\nDataset loaded from: {data_root}")
    except Exception as e:
        print(f"Failed to download dataset: {e}")
        print("Please run: pip install kagglehub")
        exit(1)

    print("\nPACS Domain Generalization Experiment")
    print("="*80)
    print(f"Using dataset from: {data_root}")
    print("="*80)

    # 모든 도메인에 대해 실험 수행
    all_results = {}

    for test_domain in all_domains:
        best_acc, results = run_domain_generalization(data_root, test_domain)
        if best_acc is not None:
            all_results[test_domain] = best_acc

    # 최종 결과 출력
    print("\n" + "="*80)
    print("Final Results - Domain Generalization Performance")
    print("="*80)

    if all_results:
        for domain, acc in all_results.items():
            print(f"{domain:15s}: {acc:.2f}%")

        avg_acc = sum(all_results.values()) / len(all_results)
        print("-"*80)
        print(f"{'Average':15s}: {avg_acc:.2f}%")
        print("="*80)
    else:
        print("No results available. Please check your data path.")
        print("\nTroubleshooting:")
        print("1. Make sure kagglehub is installed: pip install kagglehub")
        print("2. Check if Kaggle authentication is set up properly")
        print("3. Verify dataset path structure")

Using Colab cache for faster access to the 'pacs-dataset' dataset.
Path to dataset files: /kaggle/input/pacs-dataset

Dataset loaded from: /kaggle/input/pacs-dataset

PACS Domain Generalization Experiment
Using dataset from: /kaggle/input/pacs-dataset

Testing on domain: photo
Training on domains: ['art_painting', 'cartoon', 'sketch']

Found domain 'art_painting' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/art_painting
Found domain 'cartoon' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/cartoon
Found domain 'sketch' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/sketch
Loaded 8321 samples from domains: ['art_painting', 'cartoon', 'sketch']
Found domain 'photo' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/photo
Loaded 1670 samples from domains: ['photo']

Epoch 1/30
------------------------------------------------------------


Training: 100%|██████████| 260/260 [00:25<00:00, 10.01it/s, loss=2.4669, acc=23.06%]


Train Loss: 2.4574, Train Acc: 23.06%
Test Loss: 1.9216, Test Acc: 21.92%
✓ Best model saved! Accuracy: 21.92%

Epoch 2/30
------------------------------------------------------------


Training: 100%|██████████| 260/260 [00:18<00:00, 14.30it/s, loss=1.7594, acc=29.76%]


Train Loss: 1.7459, Train Acc: 29.76%
Test Loss: 1.9006, Test Acc: 23.83%
✓ Best model saved! Accuracy: 23.83%

Epoch 3/30
------------------------------------------------------------


...

Train Loss: 1.2614, Train Acc: 52.41%
Test Loss: 1.8331, Test Acc: 33.93%

Epoch 29/30
------------------------------------------------------------


Training: 100%|██████████| 189/189 [00:15<00:00, 11.90it/s, loss=1.2821, acc=52.08%]


Train Loss: 1.2618, Train Acc: 52.08%
Test Loss: 1.8350, Test Acc: 34.41%

Epoch 30/30
------------------------------------------------------------


Training: 100%|██████████| 189/189 [00:15<00:00, 12.02it/s, loss=1.2753, acc=51.70%]
                                                             

Train Loss: 1.2686, Train Acc: 51.70%
Test Loss: 1.8947, Test Acc: 32.73%

Final Results - Domain Generalization Performance
photo          : 35.75%
art_painting   : 26.86%
cartoon        : 34.22%
sketch         : 36.19%
--------------------------------------------------------------------------------
Average        : 33.25%
